In [1]:
import json
import sh
import re
import glob
from pathlib import Path
import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from collections import defaultdict

import analysishelper as ah
import cachedefs as cd

plt.rcParams['pdf.fonttype'] = 42
plt.rcParams.update({'mathtext.default':  'regular' })

In [2]:
def get_all_appids():
    with open("../../download/02_daily-profile-download/apps.csv", "r") as f:
        return [line.split(',')[0].strip() for line in f.readlines()]

def get_scrapedays():
    return [d.name for d in ah.daily_dirs()]

In [3]:

def old_gen_df_versions():
    # this is too slow, we change the dexmetadata cache and compute it again
    data = {}
    for appid, day in tqdm.tqdm([(appid, day) for appid in get_all_appids() for day in get_scrapedays()]):
        if appid not in data.keys():
            data[appid] = {}
        apkdir = ah.ALL_DAILY_DIR / day / appid
        data[appid][day] = "-1"
        if ah.get_base_apkpath(apkdir).exists():
            dexmeta = ah.cached_apk_to_version(apkdir)
            if dexmeta:
                data[appid][day] = dexmeta['versionCode'].strip()
    return pd.DataFrame.from_dict(data).sort_index(ascending=True)

def gen_df_versions():
    data = {}
    for appid in get_all_appids():
        data[appid] = {}
        for day in get_scrapedays():
            data[appid][day] = "-1"
    ah.setup_hashshelve()
    try:
        for apkdir in tqdm.tqdm([
                adir 
                for daily_dir in ah.daily_dirs()
                for adir in ah.apkdirs_from_daily_dir(daily_dir)
            ]):
            date, apkid = ah.get_date_apkid(apkdir)
            basepath = ah.get_base_apkpath(apkdir)
            # we load this directly - if the cachefile does not exist we don't have the result
            # if the cachefile exists it only has the earliest version code, so we can't iterate over just that 
            try:
                apk_hash = ah.get_filehash(basepath)
            except sh.ErrorReturnCode_1:
                continue
            cachefile = ah.APK_VERSION_HASH_CACHE / apk_hash
            if cachefile.exists():
                with open(cachefile) as f:
                    dexmeta = json.load(f)
                    if dexmeta:
                        versionCode = dexmeta['versionCode']
                        data[apkid][date] = versionCode
    finally:
        ah.close_hashshelve()
    
    return pd.DataFrame.from_dict(data).sort_index(ascending=True)


def get_df_versions():
    if cd.CACHE_DF_VERSIONS_PATH.exists():
        print("read from cache")
        return pd.read_pickle(cd.CACHE_DF_VERSIONS_PATH)
    else:
        df_versions = gen_df_versions()
        df_versions.to_pickle(cd.CACHE_DF_VERSIONS_PATH)
        return df_versions

df_versions = get_df_versions()


read from cache


In [4]:
df_versions

,com.google.android.webview,com.android.chrome,com.facebook.katana,com.google.android.tts,com.google.android.googlequicksearchbox,com.google.android.apps.restore,com.whatsapp,com.google.android.inputmethod.latin,com.google.android.apps.docs,com.google.android.marvin.talkback,...,com.imo.android.imoimhd,com.tokopedia.tkpd,com.netqin.ps,com.ht.mini.car.raceway.endless.drive,com.crazylabs.sausage.run,air.WR3DFree,com.redforcegames.stack.colors,com.psafe.msuite,net.mbc.shahid,com.gameloft.android.ANMP.GloftFWHM
2025-03-31,UNKNOWN,UNKNOWN,460214426,210547004,301489443,738103,250884004,162032162,213946996,60145309,...,25011088,320330501,331,622,1303,1730064,45356,-1,-1,750216
2025-04-01,UNKNOWN,UNKNOWN,460214400,210547004,301489443,738103,250884004,163542538,213946996,60145309,...,25011088,320330501,331,622,1303,1730064,45356,-1,-1,750216
2025-04-02,UNKNOWN,-1,460214426,210547004,301489683,738103,250884004,163542538,213946996,60145309,...,25011088,320330501,331,622,1303,1730064,45356,-1,-1,750216
2025-04-03,UNKNOWN,UNKNOWN,460416309,210547004,301489443,738103,250884004,163542538,213946996,60145309,...,25011088,320330501,331,622,1303,1730064,45356,-1,-1,750216
2025-04-04,-1,UNKNOWN,460416333,210547004,301492995,738103,250978004,163542538,213946996,60145309,...,25011088,320330501,331,622,1303,1730064,45356,-1,-1,750216
2025-04-05,UNKNOWN,UNKNOWN,460416333,210547004,301492995,738103,250978004,163542538,213946996,60145309,...,25011088,320330501,331,622,1303,1730064,45356,-1,-1,750216
2025-04-06,UNKNOWN,UNKNOWN,460416333,210547004,301492995,738103,250978004,163542538,213946996,60145309,...,25011088,320330501,331,622,1303,1730064,45356,-1,-1,750216
2025-04-07,UNKNOWN,UNKNOWN,460416212,210547004,301492995,-1,250978004,163542538,213946996,60145309,...,25011088,320330501,331,622,1303,1730064,45356,-1,-1,750216
2025-04-08,UNKNOWN,UNKNOWN,460416269,210547004,301492995,738103,250978004,163542538,213947108,60150381,...,25011088,320330501,331,622,1303,1730064,45356,-1,-1,750216
2025-04-09,UNKNOWN,UNKNOWN,-1,210547004,301492995,766453,250978004,163542538,213946996,60150381,...,25021108,320330501,331,622,1304,1730064,45356,-1,-1,750216


In [5]:
def old_gen_df_cloudhashes():
    data = {}
    for appid in tqdm.tqdm(get_all_appids()):
        data[appid] = {}
        for day in get_scrapedays():
            data[appid][day] = -1
            primary_prof_path = DAILY_PROFILES_PATH / day / appid / "primary.prof" 
            if primary_prof_path.exists():
                with open(primary_prof_path, "r", errors='backslashreplace') as f:
                    data[appid][day] = hash(str(f.read()))
    return pd.DataFrame.from_dict(data).sort_index(ascending=True)

def gen_df_cloudhashes():
    data = {}
    for appid in get_all_appids():
        data[appid] = {}
        for day in get_scrapedays():
            data[appid][day] = "-1"
    for apkdir in tqdm.tqdm([
            adir 
            for daily_dir in ah.daily_dirs()
            for adir in ah.apkdirs_from_daily_dir(daily_dir)
        ]):
        date, apkid = ah.get_date_apkid(apkdir)
        basepath = ah.get_base_apkpath(apkdir)
        # same as above, if we use the cached access through ah we try to regen it if its not available. don't want that
        # for some reason hashelve has some wrong entries here. we do not use it
        targetdir = ah.get_cache_profile_cloud_dir(apkdir)
        cloudprof = targetdir / ah.IN_DM_CLOUDPROFILE_PATH
        if not cloudprof.exists():
            continue
        cloudhash = ah.get_filehash(cloudprof)
        data[apkid][date] = int(str(cloudhash).strip(), 16)
        #profdump = ah.PROF_HASH_CACHE / cloudhash
        #if profdump.exists():
        #    with open(profdump, "r", errors='backslashreplace') as f:
        #        data[apkid][date] = hash(str(f.read()))
    
    return pd.DataFrame.from_dict(data).sort_index(ascending=True)

def get_df_cloudhashes():
    if cd.CACHE_DF_CLOUDHASHES_PATH.exists():
        return pd.read_pickle(cd.CACHE_DF_CLOUDHASHES_PATH)
    else:
        df_cloudhashes = gen_df_cloudhashes()
        df_cloudhashes.to_pickle(cd.CACHE_DF_CLOUDHASHES_PATH)
        return df_cloudhashes
    
df_cloudhashes = get_df_cloudhashes()

In [6]:
df_cloudhashes

,com.google.android.webview,com.android.chrome,com.facebook.katana,com.google.android.tts,com.google.android.googlequicksearchbox,com.google.android.apps.restore,com.whatsapp,com.google.android.inputmethod.latin,com.google.android.apps.docs,com.google.android.marvin.talkback,...,com.imo.android.imoimhd,com.tokopedia.tkpd,com.netqin.ps,com.ht.mini.car.raceway.endless.drive,com.crazylabs.sausage.run,air.WR3DFree,com.redforcegames.stack.colors,com.psafe.msuite,net.mbc.shahid,com.gameloft.android.ANMP.GloftFWHM
2025-03-31,6901447238704842522649131035497137001813037568...,6447054887666572025636670207796865504008652045...,2354917045622606883217133078134314652746598750...,5050671748509528223214151170828303400418137969...,1116905397271363251113404506518316130356442888...,1103715348654645993628010610192731199341761185...,2175887187181980196857172827517034979184600690...,1694295861639019410719563062696927614412729774...,9244199803596992185511176664592117641834919646...,5038339117441867624851186497703142612300069784...,...,9851101694805825558791887126150299915072985530...,3558183924569252109840769239383754034959971541...,1496725685868527127075038694976225540601976779...,1130618322524526978306268837559497688948758265...,1686587877732577753762310579419574375350786610...,8374703005417944627071409781604214943472436478...,1942465463777067517516276375361527105413575155...,-1,3478984546929826373174908693303710815328340386...,7404417736584053108739969348827675861132669503...
2025-04-01,6901447238704842522649131035497137001813037568...,6447054887666572025636670207796865504008652045...,9138746699850773938848568032229418893144254424...,5050671748509528223214151170828303400418137969...,1116905397271363251113404506518316130356442888...,1103715348654645993628010610192731199341761185...,2175887187181980196857172827517034979184600690...,8668324364978575195017038897899210381722082045...,9244199803596992185511176664592117641834919646...,5038339117441867624851186497703142612300069784...,...,9851101694805825558791887126150299915072985530...,3558183924569252109840769239383754034959971541...,1496725685868527127075038694976225540601976779...,1130618322524526978306268837559497688948758265...,1686587877732577753762310579419574375350786610...,8374703005417944627071409781604214943472436478...,1942465463777067517516276375361527105413575155...,-1,3478984546929826373174908693303710815328340386...,7404417736584053108739969348827675861132669503...
2025-04-02,6901447238704842522649131035497137001813037568...,-1,2354917045622606883217133078134314652746598750...,5050671748509528223214151170828303400418137969...,7075621351160890442166543232457073158944353103...,1103715348654645993628010610192731199341761185...,2175887187181980196857172827517034979184600690...,8668324364978575195017038897899210381722082045...,9244199803596992185511176664592117641834919646...,5038339117441867624851186497703142612300069784...,...,9851101694805825558791887126150299915072985530...,3558183924569252109840769239383754034959971541...,1496725685868527127075038694976225540601976779...,1130618322524526978306268837559497688948758265...,1686587877732577753762310579419574375350786610...,8374703005417944627071409781604214943472436478...,1942465463777067517516276375361527105413575155...,-1,3478984546929826373174908693303710815328340386...,7404417736584053108739969348827675861132669503...
2025-04-03,6901447238704842522649131035497137001813037568...,6447054887666572025636670207796865504008652045...,2696433081934917463066939010829195509708022749...,5050671748509528223214151170828303400418137969...,1116905397271363251113404506518316130356442888...,1103715348654645993628010610192731199341761185...,2175887187181980196857172827517034979184600690...,8668324364978575195017038897899210381722082045...,9244199803596992185511176664592117641834919646...,5038339117441867624851186497703142612300069784...,...,9851101694805825558791887126150299915072985530...,3558183924569252109840769239383754034959971541...,149672568586852

In [24]:
# how fast are cloudprofiles updated after app update?
    
# how many versions of cloudprofiles for an app version?
def cloudprofs_per_version():
    affected_apps = []
    appidversion_to_cloudhashes = defaultdict(set)

    appversion_to_day_map = {}
    for appid in df_versions.columns:
        if appid not in df_cloudhashes.columns:
            print(f"WARNING: {appid} not in cloudhashes")
            continue
        versions = df_versions[appid]
        cloudhashes = df_cloudhashes[appid]
        for day, version in versions.items():
            cloudhash = cloudhashes[day]
            if version == "-1" or cloudhash == -1 or cloudhash == "-1":
                continue
            appidversion_to_cloudhashes[f"{appid}_{version}"].add(cloudhash)
            appversion_to_day_map[f"{appid}_{version}"] = day
    
    num_versions = []
    occurences_per_num_profiles = defaultdict(int)
    for appver, cloudhashes in appidversion_to_cloudhashes.items():
        num_profiles_per_app_version = len(cloudhashes)
        occurences_per_num_profiles[num_profiles_per_app_version] += 1

        if num_profiles_per_app_version > 1:
            splt = appver.split('_')
            affected_apps.append(('_'.join(splt[:-1]), splt[-1]))
            print(appver, num_profiles_per_app_version)
            print(appversion_to_day_map[appver])

        num_versions.append(num_profiles_per_app_version)

    print()
    #print(pd.DataFrame(num_versions).describe())
    print(occurences_per_num_profiles)
    return affected_apps, appidversion_to_cloudhashes
    
affected_apps, appidversion_to_cloudhashes = cloudprofs_per_version()

com.google.android.webview_UNKNOWN 10
2025-05-20
com.android.chrome_UNKNOWN 8
2025-05-20
com.google.android.youtube_1554509248 2
2025-05-20
com.instagram.android_378011820 2
2025-04-17
jp.naver.line.android_150500328 2
2025-04-09
jp.naver.line.android_150530367 2
2025-04-24
com.kiloo.subwaysurf_81689 2
2025-05-20
com.king.candycrushsaga_13020011 2
2025-05-19
cn.wps.moffice_eng_1531 2
2025-04-29
cn.wps.moffice_eng_1533 2
2025-05-19
com.einnovation.temu_34608 2
2025-04-05
com.einnovation.temu_35100 2
2025-04-22
com.linecorp.b612.android_91140111 2
2025-05-20
com.zzkko_2118 2
2025-05-20
com.shaiban.audioplayer.mplayer_1070050002 2
2025-05-20
downloader.vitmate.downloaderapp_42 2
2025-05-20
com.lyrebirdstudio.collage_304173 2
2025-04-07
ins.story.unfold_811 2
2025-05-20
com.ismaker.android.simsimi_946 2
2025-04-17
com.bandlab.bandlab_10980300 2
2025-05-20
com.tocaboca.tocalifeworld_87149 2
2025-05-20
com.airbnb.android_28010091 2
2025-05-15
com.king.candycrushjellysaga_303700000 2
2025-05-

- chrome(9 cloudprofs for a version) and webview (10) are an outliers because they are stdapps that can't be uninstalled and they break apkanalyzer
- 2867 appversions have 1 cloudprofile
- 83 have two versions of a cloudprofile
    - so 85 total


In [26]:
len(appidversion_to_cloudhashes.keys())

2952

In [8]:
unique_appids = set()

for affected in affected_apps:
    appid, version = affected
    unique_appids.add(appid)
    if not appid.startswith("com.google") and not appid.startswith("com.android"):
        print(appid, version)

com.instagram.android 378011820
jp.naver.line.android 150500328
jp.naver.line.android 150530367
com.kiloo.subwaysurf 81689
com.king.candycrushsaga 13020011
cn.wps.moffice_eng 1531
cn.wps.moffice_eng 1533
com.einnovation.temu 34608
com.einnovation.temu 35100
com.linecorp.b612.android 91140111
com.zzkko 2118
com.shaiban.audioplayer.mplayer 1070050002
downloader.vitmate.downloaderapp 42
com.lyrebirdstudio.collage 304173
ins.story.unfold 811
com.ismaker.android.simsimi 946
com.bandlab.bandlab 10980300
com.tocaboca.tocalifeworld 87149
com.airbnb.android 28010091
com.king.candycrushjellysaga 303700000
com.frontrow.vlog 6029
com.mt.mtxx.mtxx 110602
com.blacklightsw.ludo 3640
com.gameloft.android.ANMP.GloftGGHM 82035
com.graymatrix.did 203312482
com.atpc 20250518
com.openworldactiongames.ropehero.crime.city 10190001
com.halfbrick.jetpackjoyride 785196000
com.halfbrick.jetpackjoyride 787255000
com.halfbrick.jetpackjoyride 790092000
com.xvideostudio.videoeditor 1816
com.xvideostudio.videoeditor 

In [9]:
print(len(unique_appids))
for a in unique_appids:
    print(a)

64
com.shaiban.audioplayer.mplayer
com.litatom.app
com.linecorp.b612.android
com.zzkko
ins.story.unfold
com.instagram.android
com.tocaboca.tocalifeworld
com.instabridge.android
com.atpc
com.openworldactiongames.ropehero.crime.city
com.mt.mtxx.mtxx
com.gameloft.android.ANMP.GloftDOHM
com.BallGames.Woodturning
com.midasplayer.apps.bubblewitchsaga2
com.disney.frozensaga_goo
com.ismaker.android.simsimi
com.ironmeta.security.turbo.proxy.vpntomato.pro
com.global.foodpanda.android
com.myyearbook.m
com.launcher.ios11.iphonex
downloader.vitmate.downloaderapp
roll.unblock.ball.block.puzzle
eu.nordeus.topeleven.android
com.comuto
com.gokids.transportbuilding
eu.inlogic.footballcup2018.gplay
jp.naver.line.android
com.king.bubblewitch3
com.halfbrick.jetpackjoyride
com.samsung.android.service.health
com.bandlab.bandlab
com.mate.vpn
com.bitmango.rolltheballunrollme
com.airbnb.android
com.king.candycrushjellysaga
ru.ok.android
com.google.android.webview
com.nordcurrent.canteenhd
me.mycake
cn.wps.moffi

to look up diffs between the base profile and the cloud profile use:
```sh
DATE=2025-04-16; APKID=com.instagram.android; BASEP=`./analysishelper.py --show-profdump "$APKID,$DATE,False" | head -1`; CLOUDP=`./analysishelper.py --show-profdump "$APKID,$DATE,True" | head -1`; git diff -U0 --word-diff --no-index -- $BASEP $CLOUDP
```
why do this? because the cloudprofile might be just a renamed baseline profile and we want to check this

In [10]:
def show_version_cloudprofile(appid):
    print(df_versions[[appid]].merge(df_cloudhashes[[appid]], left_index=True, right_index=True).set_axis(['version', 'cloudhash'], axis=1))

In [11]:
show_version_cloudprofile("com.instagram.android") # 378011820

              version                                          cloudhash
2025-03-31  377711262  3882799382412846344715953329764237977548497241...
2025-04-01  377811769  1022778963669678007584452236374938168020840212...
2025-04-02  377811797  4622189191015225470474864765262423969687685830...
2025-04-03  377811853  7636386298224538286891576398253401726961430310...
2025-04-04  377811853  7636386298224538286891576398253401726961430310...
2025-04-05  377811769  1022778963669678007584452236374938168020840212...
2025-04-06  377811853  7636386298224538286891576398253401726961430310...
2025-04-07  377811797  4622189191015225470474864765262423969687685830...
2025-04-08  377911424  1762590941588212664782591782047712176038114976...
2025-04-09  377911424  1762590941588212664782591782047712176038114976...
2025-04-10  377911424  1762590941588212664782591782047712176038114976...
2025-04-11  377911424  1762590941588212664782591782047712176038114976...
2025-04-12  377911424  1762590941588212664782591782

```diff
diff --git 1/_analysis_cache/hashed/profdumps/eecd25dee059ac5e0df91aaa6fc6b2231c47c29612e9a9ac4ff99d368d8680ef 2/_analysis_cache/hashed/profdumps/24ad9774a6bc6f60f0863e4b5a82984ea5c14fd4307707ecbbad36539d3efffa
index a708d6b..b143981 100644
--- 1/_analysis_cache/hashed/profdumps/eecd25dee059ac5e0df91aaa6fc6b2231c47c29612e9a9ac4ff99d368d8680ef
+++ 2/_analysis_cache/hashed/profdumps/24ad9774a6bc6f60f0863e4b5a82984ea5c14fd4307707ecbbad36539d3efffa
@@ -3 +3 @@
ProfileInfo [01[-5-]{+0+}]
@@ -5,4 +5,4 @@ ProfileInfo [015]
classes.dex [index=0] [checksum=8280b148][- [num_type_ids=35019] [num_method_ids=60822]-]
        hot methods: 0[], 6[], 7[], 8[], 12[], 13[], 15[], 16[], 19[], 20[], 21[], 23[], 24[], 26[], 31[], 33[], 34[], 35[], 37[], 38[], 42[], 46[], 47[], 49[], 50[], 52[], 53[], 54[], 55[], 56[], 60[], 61[], 62[], 63[], 64[], 67[], 69[], 75[],$
{+      startup methods: 0, 6, 7, 8, 12, 13, 15, 16, 19, 20, 23, 24, 26, 31, 33, 34, 35, 37, 38, 42, 46, 47, 49, 50, 52, 53, 54, 55, 56, 60, 61, 62, 63, 64, 67, 69, 75, 77, 79, 80, 88, 92, 93, 94, 104, 110, 113, 116, 117, 119, 123, 128, 130, 13$
[-570-]{+       post startup methods: +}0[-4-], [-57005-]{+6+}, [-5-]7[-0-]{+, 8, +}1{+2, +}1{+3+}, [-570-]15, [-570-]1[-7-]{+6+}, [-57-]{+19, 2+}0{+, +}2{+1, +}2{+3+}, [-570-]2[-6-]{+4+}, [-57030-]{+26+}, [-570-]31, [-570-]33, [-570-]34, {+3+}$
@@ -10,4 +10,4 @@ classes.dex [index=0] [checksum=8280b148] [num_type_ids=35019] [num_method_ids=6
```

looks like genuinely new profile

In [12]:
show_version_cloudprofile("roll.unblock.ball.block.puzzle") # 110 # 111 # 113 # 114 # 115 # 116

# 2025-04-14 kinda similar, just some methods missing
# playstore messing with us giving uns an old version? or rollback? idk

           version                                          cloudhash
2025-03-31     100  2076522338064629960051175895492264069220583907...
2025-04-01     100  2076522338064629960051175895492264069220583907...
2025-04-02     100  2076522338064629960051175895492264069220583907...
2025-04-03     100  2076522338064629960051175895492264069220583907...
2025-04-04     100  2076522338064629960051175895492264069220583907...
2025-04-05     100  2076522338064629960051175895492264069220583907...
2025-04-06     100  2076522338064629960051175895492264069220583907...
2025-04-07     100  2076522338064629960051175895492264069220583907...
2025-04-08     100  2076522338064629960051175895492264069220583907...
2025-04-09     100  2076522338064629960051175895492264069220583907...
2025-04-10     100  2076522338064629960051175895492264069220583907...
2025-04-11     100  2076522338064629960051175895492264069220583907...
2025-04-12     100  2076522338064629960051175895492264069220583907...
2025-04-13     100  

In [13]:
show_version_cloudprofile("com.YovoGames.hair") # 154 # 162

# first cloudprof 154 removes a couple methods and 05-02 adds a bunch

           version                                          cloudhash
2025-03-31      43  6161452982080497372855875797071712398476535997...
2025-04-01      43  6161452982080497372855875797071712398476535997...
2025-04-02      43  6161452982080497372855875797071712398476535997...
2025-04-03      43  6161452982080497372855875797071712398476535997...
2025-04-04      43  6161452982080497372855875797071712398476535997...
2025-04-05      43  6161452982080497372855875797071712398476535997...
2025-04-06      43  6161452982080497372855875797071712398476535997...
2025-04-07      43  6161452982080497372855875797071712398476535997...
2025-04-08      43  6161452982080497372855875797071712398476535997...
2025-04-09      43  6161452982080497372855875797071712398476535997...
2025-04-10      43  6161452982080497372855875797071712398476535997...
2025-04-11      43  6161452982080497372855875797071712398476535997...
2025-04-12      43  6161452982080497372855875797071712398476535997...
2025-04-13      43  

In [14]:
show_version_cloudprofile("com.xvideostudio.videoeditor") # 1816 # 1817 # 1821
# first less then more in diff

           version                                          cloudhash
2025-03-31    1815  4549291970349111730132428206286712305329373628...
2025-04-01    1815  4549291970349111730132428206286712305329373628...
2025-04-02    1815  4549291970349111730132428206286712305329373628...
2025-04-03    1815  4549291970349111730132428206286712305329373628...
2025-04-04    1815  4549291970349111730132428206286712305329373628...
2025-04-05    1815  4549291970349111730132428206286712305329373628...
2025-04-06    1815  4549291970349111730132428206286712305329373628...
2025-04-07    1815  4549291970349111730132428206286712305329373628...
2025-04-08    1815  4549291970349111730132428206286712305329373628...
2025-04-09    1815  4549291970349111730132428206286712305329373628...
2025-04-10    1815  4549291970349111730132428206286712305329373628...
2025-04-11    1815  4549291970349111730132428206286712305329373628...
2025-04-12    1815  4549291970349111730132428206286712305329373628...
2025-04-13    1815  

In [15]:
show_version_cloudprofile("com.lyrebirdstudio.collage") # 304173
# first less then more in diff

           version                                          cloudhash
2025-03-31  304173  8160388892838505173584760786304492574338204822...
2025-04-01  304173  3683027857591112465846783693630640834694841412...
2025-04-02  304173  3683027857591112465846783693630640834694841412...
2025-04-03  304173  3683027857591112465846783693630640834694841412...
2025-04-04  304173  3683027857591112465846783693630640834694841412...
2025-04-05  304173  3683027857591112465846783693630640834694841412...
2025-04-06  304173  3683027857591112465846783693630640834694841412...
2025-04-07  304173  3683027857591112465846783693630640834694841412...
2025-04-08  304176  1110165528302356366089234030426148472744182076...
2025-04-09  304176  1110165528302356366089234030426148472744182076...
2025-04-10  304176  1110165528302356366089234030426148472744182076...
2025-04-11  304176  1110165528302356366089234030426148472744182076...
2025-04-12  304176  1110165528302356366089234030426148472744182076...
2025-04-13  304176  

In [16]:
show_version_cloudprofile("ins.story.unfold") # 811
# first less, then lots of changes

           version                                          cloudhash
2025-03-31     806  5071517097400356737024429318215194201538978973...
2025-04-01     806  5071517097400356737024429318215194201538978973...
2025-04-02     806  5071517097400356737024429318215194201538978973...
2025-04-03     806  5071517097400356737024429318215194201538978973...
2025-04-04     806  5071517097400356737024429318215194201538978973...
2025-04-05     806  5071517097400356737024429318215194201538978973...
2025-04-06     806  5071517097400356737024429318215194201538978973...
2025-04-07     806  5071517097400356737024429318215194201538978973...
2025-04-08     806  5071517097400356737024429318215194201538978973...
2025-04-09     806  5071517097400356737024429318215194201538978973...
2025-04-10     806  5071517097400356737024429318215194201538978973...
2025-04-11     806  5071517097400356737024429318215194201538978973...
2025-04-12     806  5071517097400356737024429318215194201538978973...
2025-04-13     806  

In [17]:
show_version_cloudprofile("com.sega.sonicdash") # 1725266350
# again: first some removals compared to baseline, then lots of additions a day later

               version                                          cloudhash
2025-03-31  1725198510  8880367424625967761016010065070201829509703690...
2025-04-01  1725198510  8880367424625967761016010065070201829509703690...
2025-04-02  1725198510  8880367424625967761016010065070201829509703690...
2025-04-03  1725198510  8880367424625967761016010065070201829509703690...
2025-04-04  1725198510  8880367424625967761016010065070201829509703690...
2025-04-05  1725198510  8880367424625967761016010065070201829509703690...
2025-04-06  1725198510  8880367424625967761016010065070201829509703690...
2025-04-07  1725198510  8880367424625967761016010065070201829509703690...
2025-04-08  1725198510  8880367424625967761016010065070201829509703690...
2025-04-09  1725198510  8880367424625967761016010065070201829509703690...
2025-04-10  1725266350  4653581074337764070600470201695587652926460873...
2025-04-11  1725266350  4994450932015588112413760023167231864175914647...
2025-04-12  1725266350  49944509320155

```py
show_version_cloudprofile("com.instagram.android") # 378011820
show_version_cloudprofile("jp.naver.line.android") # 150500328 # 150530367
show_version_cloudprofile("com.kiloo.subwaysurf") # 81689
show_version_cloudprofile("com.king.candycrushsaga") # 13020011
show_version_cloudprofile("cn.wps.moffice_eng") # 1531 # 1533
show_version_cloudprofile("com.einnovation.temu") # 34608 # 35100
show_version_cloudprofile("com.linecorp.b612.android") # 91140111
show_version_cloudprofile("com.zzkko") # 2118
show_version_cloudprofile("com.shaiban.audioplayer.mplayer") # 1070050002
show_version_cloudprofile("downloader.vitmate.downloaderapp") # 42
show_version_cloudprofile("com.lyrebirdstudio.collage") # 304173
show_version_cloudprofile("ins.story.unfold") # 811
show_version_cloudprofile("com.ismaker.android.simsimi") # 946
show_version_cloudprofile("com.bandlab.bandlab") # 10980300
show_version_cloudprofile("com.tocaboca.tocalifeworld") # 87149
show_version_cloudprofile("com.king.candycrushjellysaga") # 303700000
show_version_cloudprofile("com.frontrow.vlog") # 6029
show_version_cloudprofile("com.mt.mtxx.mtxx") # 110602
show_version_cloudprofile("com.blacklightsw.ludo") # 3640
show_version_cloudprofile("com.gameloft.android.ANMP.GloftGGHM") # 82035
show_version_cloudprofile("com.graymatrix.did") # 203312482
show_version_cloudprofile("com.atpc") # 20250518
show_version_cloudprofile("com.openworldactiongames.ropehero.crime.city") # 10190001
show_version_cloudprofile("com.halfbrick.jetpackjoyride") # 785196000 # 787255000 # 790092000
show_version_cloudprofile("com.gameloft.android.ANMP.GloftDOHM") # 85123
show_version_cloudprofile("me.mycake") # 306020201
show_version_cloudprofile("com.rovio.baba") # 32775491
show_version_cloudprofile("jp.naver.linecamera.android") # 197
show_version_cloudprofile("com.mate.vpn") # 49712
show_version_cloudprofile("com.BallGames.Woodturning") # 4021
show_version_cloudprofile("com.drweb") # 19134
show_version_cloudprofile("com.nordcurrent.canteenhd") # 11154
show_version_cloudprofile("com.fa.gym.fighting.game") # 200192
show_version_cloudprofile("ru.ok.android") # 25042101
show_version_cloudprofile("com.global.foodpanda.android") # 251410154
show_version_cloudprofile("eu.nordeus.topeleven.android") # 8817 # 8834
show_version_cloudprofile("free.vpn.unblock.proxy.vpn.master.pro") # 2025040711
show_version_cloudprofile("com.mastercomlimited.cardriving_t") # 614013107
show_version_cloudprofile("com.YovoGames.hair") # 154 # 162
show_version_cloudprofile("com.king.bubblewitch3") # 1000000000
show_version_cloudprofile("com.ironmeta.security.turbo.proxy.vpntomato.pro") # 28833
show_version_cloudprofile("com.tapblaze.pizzabusiness") # 1878
show_version_cloudprofile("com.tutotoons.app.fluvsies.free") # 10001213
show_version_cloudprofile("com.mobirix.fishinghook") # 250407
show_version_cloudprofile("com.midasplayer.apps.bubblewitchsaga2") # 117500000
show_version_cloudprofile("com.samsung.android.service.health") # 160201000
show_version_cloudprofile("com.launcher.ios11.iphonex") # 20250403
show_version_cloudprofile("eu.inlogic.footballcup2018.gplay") # 170
show_version_cloudprofile("com.gokids.transportbuilding") # 12005007
show_version_cloudprofile("com.bitmango.rolltheballunrollme") # 613
show_version_cloudprofile("com.disney.frozensaga_goo") # 969
show_version_cloudprofile("com.comuto") # 340000513 # 340000514
show_version_cloudprofile("com.superking.parchisi.star") # 405
show_version_cloudprofile("com.sinyee.babybus.restaurant") # 8720001
show_version_cloudprofile("com.instabridge.android") # 101615 # 101617 # 101619 # 101620 # 101621 # 101622
show_version_cloudprofile("com.sega.sonicdash") # 1725266350
show_version_cloudprofile("com.litatom.app") # 557 # 566

```

we investigated 7 manually of them and found ..

----
----
# old stuff

In [18]:
_last_day = '2024-05-20'
_first_day = '2024-03-31'

def calculate_days_since(df_iterable):
    previous_val = "-1" # missing
    days_since_update = np.nan
    data = []
    last_known = "-1"
    
    for index, val in df_iterable.iterrows():
        val = val.to_numpy()[0]
        if val == "-1":
            days_since_update = np.nan
        else:
            if previous_val == "-1":
                days_since_update = 0
            else:
                # val != -1 and previous_val !=-1
                if val == previous_val:
                    days_since_update += 1
                else:
                    
                    days_since_update = 0
        #print(index, "-" ,val)
        #print(days_since_update)
        #print()
        data.append(days_since_update)
        previous_val = val
    return data
    
def test(df):
    data = {}
    for c in df.columns:
        data[c] = calculate_days_since(df[[c]])
    return data

days_since_cloudupdate = test(df_cloudhashes)
days_since_appupdate = test(df_versions)


In [19]:
pd.DataFrame.from_dict(days_since_cloudupdate).astype(pd.Int32Dtype()).dropna(axis=1)

,com.google.android.tts,com.whatsapp,com.google.android.inputmethod.latin,com.google.android.apps.docs,com.google.android.marvin.talkback,com.google.android.gms,com.google.android.apps.photos,com.google.android.apps.wellbeing,com.google.android.apps.youtube.music,com.instagram.android,...,com.myyearbook.m,com.imo.android.imoimhd,com.tokopedia.tkpd,com.netqin.ps,com.ht.mini.car.raceway.endless.drive,com.crazylabs.sausage.run,air.WR3DFree,com.redforcegames.stack.colors,net.mbc.shahid,com.gameloft.android.ANMP.GloftFWHM
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,0,1,1,1,1,1,1,0,...,1,1,1,1,1,1,1,1,1,1
2,2,2,1,2,2,2,0,2,2,0,...,2,2,2,2,2,2,2,2,2,2
3,3,3,2,3,3,3,1,3,3,0,...,3,3,3,3,3,3,3,3,3,3
4,4,0,3,4,4,0,2,4,0,1,...,4,4,4,4,4,4,4,4,4,4
5,5,1,4,5,5,1,3,5,1,0,...,5,5,5,5,5,5,5,5,5,5
6,6,2,5,6,6,2,4,6,2,0,...,6,6,6,6,6,6,6,6,6,6
7,7,3,6,7,7,3,5,7,3,0,...,7,7,7,7,7,7,7,7,0,7
8,8,4,7,0,0,4,6,0,4,0,...,8,8,8,8,8,8,8,8,1,8
9,9,5,8,0,1,5,0,0,5,1,...,9,0,9,9,9,0,9,9,2,9


In [20]:
df_days_since_cloudupdate = pd.DataFrame.from_dict(days_since_cloudupdate).astype(pd.Int32Dtype()).dropna(axis=1)
df_days_since_appupdate = pd.DataFrame.from_dict(days_since_appupdate).astype(pd.Int32Dtype()).dropna(axis=1)

In [21]:
df_days_since_appupdate

,com.google.android.tts,com.whatsapp,com.google.android.inputmethod.latin,com.google.android.apps.docs,com.google.android.marvin.talkback,com.google.android.gms,com.google.android.apps.photos,com.google.android.apps.wellbeing,com.google.android.apps.youtube.music,com.instagram.android,...,com.talpa.hibrowser,com.myyearbook.m,com.imo.android.imoimhd,com.tokopedia.tkpd,com.netqin.ps,com.ht.mini.car.raceway.endless.drive,com.crazylabs.sausage.run,air.WR3DFree,com.redforcegames.stack.colors,com.gameloft.android.ANMP.GloftFWHM
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,0,1,1,1,1,1,1,0,...,1,1,1,1,1,1,1,1,1,1
2,2,2,1,2,2,2,0,2,2,0,...,2,2,2,2,2,2,2,2,2,2
3,3,3,2,3,3,3,1,3,3,0,...,3,3,3,3,3,3,3,3,3,3
4,4,0,3,4,4,0,2,4,0,1,...,4,4,4,4,4,4,4,4,4,4
5,5,1,4,5,5,1,3,5,1,0,...,5,5,5,5,5,5,5,5,5,5
6,6,2,5,6,6,2,4,6,2,0,...,6,6,6,6,6,6,6,6,6,6
7,7,3,6,7,7,3,5,7,3,0,...,7,7,7,7,7,7,7,7,7,7
8,8,4,7,0,0,4,6,0,4,0,...,8,8,8,8,8,8,8,8,8,8
9,9,5,8,0,1,5,0,0,5,1,...,9,9,0,9,9,9,0,9,9,9


In [22]:
df_updates = df_days_since_appupdate.eq(0).sum()

df_updates.describe()

count       792.0
mean     3.844697
std      3.923075
min           1.0
25%           1.0
50%           3.0
75%           5.0
max          33.0
dtype: Float64

In [23]:
df_at_least_one_update = df_updates.loc[~(df_updates==1)] -1
df_at_least_one_update.describe()

# 

count       538.0
mean     4.187732
std      4.127403
min           1.0
25%           2.0
50%           3.0
75%           6.0
max          32.0
dtype: Float64